# Topic analysis

Compare the word frequencies in one group of tweets with the word frequencies in another group of tweets to find out the topics discussed in the tweets.

In [5]:
import csv
import math
import os
import pandas as pd
import re
import sys
from IPython.display import clear_output
from nltk.tokenize import TweetTokenizer
sys.path.append("/home/erikt/projects/newsgac/fasttext-runs")
import tscore

In [19]:
DATADIR = "/home/erikt/projects/puregome/data/text/"
IDSTR = "id_str"
NBROFEXAMPLES = 20
TEXT = "text"
TOKEN = "token"
TOKENFILE = "tokens.csv"
USER = "user"

In [20]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)
        
def tokenize(text):
    return(TweetTokenizer().tokenize(text))

def isInterestingToken(token):
    return(re.search(r"[a-z]",token) and not re.search("http",token))

def getTokenCountsFromTweets(datePattern,query="",baseDict={}):
    tokenCounts = dict(baseDict)
    fileList = sorted(os.listdir(DATADIR))
    for inFileName in fileList:
        if re.search(datePattern,inFileName):
            squeal(inFileName)
            df = pd.read_csv(DATADIR+inFileName,index_col=IDSTR)
            for i in range(0,len(df)):
                text = df.iloc[i][TEXT]
                if re.search(query,text):
                    for token in set(tokenize(text.lower())): 
                        if isInterestingToken(token):
                            if not token in tokenCounts: tokenCounts[token] = 0
                            tokenCounts[token] += 1
    return(tokenCounts)

def dictTopN(dictionary,N=NBROFEXAMPLES):
    return([(x[1],x[0]) for x in dictionary.items()][0:N])

In [13]:
NBROFTOKENS = "totalFreq"
NBROFTYPES = "nbrOfWords"
WORDFREQS = "wordFreqs"

def makeTscoreData(tokenList):
    data = { NBROFTOKENS:0, NBROFTYPES:0, WORDFREQS:{} }
    for token in tokenList:
        if not math.isnan(tokenList[token]):
            data[WORDFREQS][token] = tokenList[token]
            data[NBROFTYPES] += 1
            data[NBROFTOKENS] += tokenList[token]
    return(data)

def sortTscores(tscores):
    return({token:tscores[token] \
            for token in sorted(tscores.keys(),key=lambda t:tscores[t],reverse=True)})

In [30]:
#tokenCounts1 = getTokenCountsFromTweets("20200531")
#tokenCounts2 = getTokenCountsFromTweets("2020060[123]")
tokenCounts20200614 = getTokenCountsFromTweets("20200614")

20200614-23.out.gz


In [32]:
#tscores1 = makeTscoreData(tokenCounts1)
tscores2 = makeTscoreData(tokenCounts20200614)
dictTopN(sortTscores(tscore.computeTscore(tscores2,tscores1)))

[(78.17528811133893, '@thierrybaudet'),
 (74.74164710863714, 'hoofddoek'),
 (67.81095034059273, '@hugodejonge'),
 (61.47989588957057, '#touchepasamesetudes'),
 (56.09220632070616, '#saneerdenpo'),
 (55.513761179576974, '#banliveexports'),
 (53.247873538299054, '#stopverreveetransporten'),
 (50.57858479473896, '@simoneweimans'),
 (49.87117962415461, 'vaderdag'),
 (46.428143094114425, 'excuses'),
 (44.10278844138068, '#7dag'),
 (42.93923587128512, '#pgbindeknel'),
 (42.923146528257185, 'beeldenstorm'),
 (42.03637706111756, '@laurensvhg'),
 (41.786972524335305, 'baudet'),
 (41.323649272594416, '#blacklivesmatternl'),
 (41.15761405310331, '@celalaltuntas1'),
 (41.088821054676735, '@thiddema'),
 (40.92146201635439, 'londen'),
 (40.183144952753054, 'discriminatie')]

## Old code

In [3]:
DATEPATTERN = "2020031[1245]"
QUERY = "corona|covid|flattenthecurve|blijfthuis|rivm|mondkapje|huisarts|houvol|zorg"
NBROFEXAMPLES = 10

def getExamples(datePattern,query1=QUERY,query2=""):
    fileList = sorted(os.listdir(DATADIR))
    tweets = {}
    for inFileName in fileList:
        if re.search(datePattern,inFileName):
            clear_output(wait=True)
            print(inFileName)
            df = pd.read_csv(DATADIR+inFileName,compression="gzip",index_col=ID)
            for i in range(0,len(df)):
                text = df.iloc[i][TEXT]
                if re.search(query1,text) and re.search(query2,text):
                    if text in tweets: tweets[text] += 1
                    else: tweets[text] = 1
    return({tweet:tweets[tweet] for tweet in sorted(tweets.keys(),key=lambda t:tweets[t],reverse=True)})

def getTokenCountsFromTweets(datePattern,query=""):
    tokenCounts = {}
    fileList = sorted(os.listdir(DATADIR))
    for inFileName in fileList:
        if re.search(datePattern,inFileName):
            squeal(inFileName)
            df = pd.read_csv(DATADIR+inFileName,index_col=IDSTR)
            for i in range(0,len(df)):
                text = df.iloc[i][TEXT]
                if re.search(query,text):
                    for token in set(tokenize(text.lower())): 
                        if re.search(r"[a-z]",token):
                            if not token in tokenCounts: tokenCounts[token] = 0
                            tokenCounts[token] += 1
    return(tokenCounts)

def writeData(data,fileName):
    pd.DataFrame(data).to_csv(fileName,index_label=TOKEN)
    
def readData(fileName):
    return(pd.read_csv(fileName,index_col=TOKEN).to_dict())

def dictTopN(dictionary,N=NBROFEXAMPLES):
    return([(x[1],x[0]) for x in dictionary.items()][0:N])

In [6]:
#tokens = readData(TOKENFILE)
for month in "202003 202004 202005".split():
    print(month)
    tokens = makeData(month)
    writeData(tokens,"tokens"+month+".csv")

20200525-18.out.gz


In [59]:
tscoreData = {}
for date in tokens:
    tscoreData[date] = makeTscoreData(tokens[date])

In [89]:
dictTopN(sortTscores(tscore.computeTscore(tscoreData["20200312"],tscoreData["20200311"])),N=20)

[(91.60442025764918, '#coronadebat'),
 (78.97741163967773, '#covid_19'),
 (65.5129728502599, 'scholen'),
 (58.05318808343848, 'rutte'),
 (45.94871916226014, 'kinderen'),
 (42.2056255747426, 'kabinet'),
 (41.77929963688309, 'maatregelen'),
 (40.13852933881494, 'trump'),
 (38.934780261956206, 'onderwijs'),
 (38.143346719667484, 'sluiten'),
 (36.15941845414344, 'ouders'),
 (34.67443174873152, '#persconferentie'),
 (32.30460846375055, 'schouders'),
 (31.94561264372927, 'vanavond'),
 (31.443044262387314, 'persconferentie'),
 (30.64989800427607, '#coronahulp'),
 (30.451036528043275, 'vvd'),
 (30.242371033416283, 'blijven'),
 (30.154322917933154, 'onderwijspersoneel'),
 (30.12977279038019, '#coronavirusnederland')]

In [90]:
dictTopN(sortTscores(tscore.computeTscore(tscoreData["20200315"],tscoreData["20200314"])),N=20)

[(56.10373202523538, '#coronavirusnl'),
 (51.27541386261536, '16u'),
 (43.506825050815195, 'horeca'),
 (42.68413740392834, '#covid19nl'),
 (42.666377863812585, 'genomen'),
 (42.130617286003975, '#lockdownnl'),
 (41.27300848805268, 'april'),
 (38.706021588851996, 'morgenavond'),
 (37.41640267762352, 'nbelgen'),
 (36.594544941735116, 'aanvullende'),
 (36.34116105725093, 'fuiven'),
 (36.13170079804637, 'pletter'),
 (35.97562853910354, 'beginnen'),
 (35.90013496104832, 'uitlachen'),
 (35.520520050898924, 'morgenvroeg'),
 (35.489863712482006, '9u'),
 (35.448115484855634, 'maatregelen'),
 (34.7973055808141, 'negatief'),
 (34.72120740646297, 'ndat'),
 (34.30839614997719, '@mohamedouaamari')]

## Analysis

There is a clear impact of the national press conferences on the topic tweets of the two volume peak dates. On 20200312, both the event (*persconferentie*) and the speakers (*rutte* and *kabinet*) are present in the top 20 words selected by the tscore measure. The most important topic in the tweets was the dicussion about school closures (*scholen*, *kinderen*, *onderwijs*, *sluiten*, *ouders* and *onderwijspersoneel*). On 20200315 the main topic is the closure of bars and restaurants (*horeca*). 

In [ ]:
dictTopN(getExamples("20200312",query1=QUERY,query2="scholen"))

In [ ]:
dictTopN(getExamples("20200313",query1=QUERY,query2="scholen"))